In [13]:
import pandas as pd
from icecream import ic
import spacy
import progressbar
from spacy.matcher import Matcher
# !python -m spacy download en_core_web_sm
nlp=spacy.load("en_core_web_sm")


In [14]:
data=pd.read_csv('dataframe_true_final.csv')
text=data['txt']
data=data.drop(columns=['Unnamed: 0','txt','id'])
text=text.str.replace("'","",regex=False)
text=text.str.replace("]","",regex=False)
text=text.str.replace("[","",regex=False)
text=text.str.replace(",","",regex=False)
text=text.str.replace("A ","a ",regex=False)
text=text.str.replace("And","and",regex=False)

In [15]:
newdata=pd.DataFrame(index=text.index, columns=['PERSON','NORP','GPE','ORG','JOB'])
matcher = Matcher(nlp.vocab)
pattern_nom = [[{"DEP": "compound", "OP": "*"},{"DEP": "nsubj"}]]
matcher.add("NOM", pattern_nom)

matcher_job = Matcher(nlp.vocab)
pattern_job = [[{"DEP": "attr"}]]
matcher_job.add("JOB", pattern_job)


In [16]:
wrong=set()
for i in progressbar.progressbar(range (len(text))):
  doc=nlp(text[i])
  A=matcher(doc)
  B=matcher_job(doc)
  if A!=[]:   
    newdata.loc[i]['PERSON']=doc[A[0][1]:A[0][2]].text
  else: 
    wrong.add(i)
  if B!=[]:   
    newdata.loc[i]['JOB']=doc[B[0][1]:B[0][2]].text
  else: 
    wrong.add(i)


  for col in ['NORP','GPE','ORG']:
    for ent in doc.ents:
      if ent.label_==col and newdata.iloc[i][col]!=newdata.iloc[i][col]:
        newdata.loc[i][col]=ent.text



for df in [newdata, text]:
  df.drop(wrong).reset_index().drop(columns=['index'])


100% (317268 of 317268) |################| Elapsed Time: 0:47:03 Time:  0:47:03


In [11]:
def essai(i): #tester le nouveau dataset
  ic(new.iloc[i])
  ic(text[i])
  doc=nlp(text[i])
  for ent in doc.ents:
    ic(ent.text)
    ic(ent.label_)
  for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)




In [18]:
newdata.to_csv('New_dataset.csv')